## Ex.3

Find on the Internet 5 inverted repeats in order to simulate transposes. Use these inverted repeats  to find transposomes inside a plant genoms , such as zachisa. Make a GUI that allows users to copy-paste the inverted repeats, and that shows a chart with the positions pf these transposomes over the corn genome

In [1]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
from IPython.display import display
import ipywidgets as widgets

# Common inverted repeats found in plants
DEFAULT_REPEATS = {
    "Ac/Ds": ("TAGGGATGAAA", "TTTCATCCCAT"),  # Activator/Dissociation
    "En/Spm": ("CACTATACGAA", "TTCGTATAGTG"),  # Enhancer/Suppressor-mutator
    "Mu": ("GCCTCCAATAG", "CTATTGGAGGC"),      # Mutator
    "CACTA": ("CACTACAAGAA", "TTCTTGTAGTG"),   # CACTA family
    "Helitron": ("TCTAGCTA", "TAGCTAGA")       # Helitron family
}

def find_transposons(sequence, left_ir, right_ir):
    """Find all transposon positions in the sequence"""
    positions = []
    for i in range(len(sequence)):
        if sequence[i:i+len(left_ir)] == left_ir:
            # Look for matching right IR
            for j in range(i + len(left_ir), len(sequence)):
                if sequence[j:j+len(right_ir)] == right_ir:
                    positions.append((i, j + len(right_ir)))
                    break
    return positions

def analyze_sequence(sequence, repeats):
    """Analyze sequence for transposons and create visualization"""
    fig, ax = plt.subplots(figsize=(15, 6))
    
    # Plot sequence length as reference line
    ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    
    # Find and plot transposons for each repeat pair
    colors = plt.cm.Set3(np.linspace(0, 1, len(repeats)))
    
    for i, ((name, (left_ir, right_ir)), color) in enumerate(zip(repeats.items(), colors)):
        positions = find_transposons(sequence, left_ir, right_ir)
        
        # Plot transposon positions
        for start, end in positions:
            ax.plot([start, end], [i, i], color=color, linewidth=4, label=name)
            ax.plot([start], [i], 'v', color=color)
            ax.plot([end], [i], '^', color=color)
            
            # Add position labels
            ax.text(start, i+0.2, f'Start: {start}', rotation=45, fontsize=8)
            ax.text(end, i+0.2, f'End: {end}', rotation=45, fontsize=8)
    
    ax.set_xlabel("Sequence Position")
    ax.set_ylabel("Repeat Pairs")
    ax.set_title("Transposon Positions in Sequence")
    ax.grid(True, alpha=0.3)
    
    # Add legend without duplicates
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), 
             title="Transposon Types", bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    return fig

# Create widgets
sequence_input = widgets.Textarea(
    value='',
    placeholder='Paste your DNA sequence here...',
    description='Sequence:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='100px')
)

analyze_button = widgets.Button(
    description='Analyze Sequence',
    button_style='primary',
    tooltip='Click to analyze the sequence'
)

output = widgets.Output()

def on_analyze_button_clicked(b):
    with output:
        output.clear_output()
        sequence = sequence_input.value.upper()
        if sequence:
            fig = analyze_sequence(sequence, DEFAULT_REPEATS)
            plt.show()
            
            # Print detailed results
            print("\nDetailed Results:")
            for name, (left_ir, right_ir) in DEFAULT_REPEATS.items():
                positions = find_transposons(sequence, left_ir, right_ir)
                if positions:
                    print(f"\n{name} transposons found at positions:")
                    for start, end in positions:
                        print(f"  Start: {start}, End: {end}, Length: {end-start}")
        else:
            print("Please enter a DNA sequence")

analyze_button.on_click(on_analyze_button_clicked)

# Display widgets
display(widgets.VBox([
    widgets.HTML(value="<h3>Transposon Detection Tool</h3>"),
    sequence_input,
    analyze_button,
    output
]))